In [1]:
import numpy as np
import tensorflow as tf
import deepchem as dc

/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
sider_tasks, sider_datasets, transformers = dc.molnet.load_sider(featurizer = 'GraphConv')
train_dataset, valid_dataset, test_dataset = sider_datasets

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [3]:
sider_datasets

(<deepchem.data.datasets.DiskDataset at 0x7f9a24120278>,
 <deepchem.data.datasets.DiskDataset at 0x7f99d88e6e10>)

In [4]:
train_dataset.y.shape

(1141, 27)

In [5]:
valid_dataset.y.shape

(143, 27)

In [6]:
test_dataset.y.shape

(143, 27)

In [7]:
featurizer = dc.feat.CircularFingerprint(size = 1024)

In [8]:
loader = dc.data.CSVLoader(
      tasks=sider_tasks, smiles_field="smiles",
      featurizer=featurizer)

In [9]:
dataset = loader.featurize('sider.csv')

Loading raw samples now.
shard_size: 8192
About to start loading CSV from sider.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 1.907 s
TIMING: dataset construction took 1.953 s
Loading dataset from disk.


In [10]:
sider_tasks

['Hepatobiliary disorders',
 'Metabolism and nutrition disorders',
 'Product issues',
 'Eye disorders',
 'Investigations',
 'Musculoskeletal and connective tissue disorders',
 'Gastrointestinal disorders',
 'Social circumstances',
 'Immune system disorders',
 'Reproductive system and breast disorders',
 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)',
 'General disorders and administration site conditions',
 'Endocrine disorders',
 'Surgical and medical procedures',
 'Vascular disorders',
 'Blood and lymphatic system disorders',
 'Skin and subcutaneous tissue disorders',
 'Congenital, familial and genetic disorders',
 'Infections and infestations',
 'Respiratory, thoracic and mediastinal disorders',
 'Psychiatric disorders',
 'Renal and urinary disorders',
 'Pregnancy, puerperium and perinatal conditions',
 'Ear and labyrinth disorders',
 'Cardiac disorders',
 'Nervous system disorders',
 'Injury, poisoning and procedural complications']

In [12]:
splitter = dc.splits.RandomSplitter('sider.csv')
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
    dataset)
#NOTE THE RENAMING:
valid_dataset, test_dataset = test_dataset, valid_dataset


Computing train/valid/test indices
TIMING: dataset construction took 0.064 s
Loading dataset from disk.
TIMING: dataset construction took 0.025 s
Loading dataset from disk.
TIMING: dataset construction took 0.023 s
Loading dataset from disk.


In [14]:
n_features = train_dataset.get_data_shape()[0]

1141

In [58]:
params_dict = {"activation": ["relu","sigmoid","tahn"],
               "optimizer": ["Adam","RMSprop"],
               "momentum": [.9],
               "penalty": [0.]
              }

n_features = train_dataset.get_data_shape()[0]
def model_builder(model_params, model_dir):
    model = dc.models.MultitaskClassifier(
    len(sider_tasks), n_features, **model_params)
    return model

In [59]:

metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
optimizer = dc.hyper.HyperparamOpt(model_builder)
best_dnn, best_hyperparams, all_results = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, [], metric)

Fitting model 1/6
hyperparameters: {'activation': 'relu', 'optimizer': 'Adam', 'momentum': 0.9, 'penalty': 0.0}


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Only one class present in y_true. ROC AUC score is not defined in that case.
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [0.7370384917517674, 0.6264007171671895, nan, 0.6247464503042596, 0.6358136721113128, 0.6726354453627181, 0.9300751879699248, 0.6764492753623188, 0.6373823397579561, 0.7057902973395931, 0.6634057971014493, 0.7218934911242604, 0.6384057971014492, 0.5802987861811391, 0.6748447204968944, 0.7409638554216867, 0.6933842239185751, 0.687992125984252, 0.6103723404255319, 0.6680810938236681, 0.679368932038835, 0.6739784946236559, 0.6323308270676691, 0.6630347054075867, 0.6505376344086022, 0.8581267217630854, 0.6021719858156028]
Model 1/6, Metric mean-roc_auc_score, Validation set 0: 0.680212
	best_validation_score so far: 0.680212
Fitting model 2/6
hyperparameters: {'activation': 'relu', 'optimizer': 'RMSprop', 'momentum': 0.9, 'penalty': 0.0}


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Only one class present in y_true. ROC AUC score is not defined in that case.
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [0.7421445404556166, 0.6172120125504259, nan, 0.6215010141987829, 0.6288566243194191, 0.6682736455463729, 0.91203007518797, 0.6891304347826087, 0.6149708650829224, 0.7069640062597808, 0.6735507246376812, 0.7467455621301775, 0.6474637681159421, 0.6027077497665734, 0.677639751552795, 0.7315261044176706, 0.6666666666666666, 0.6638779527559056, 0.6170212765957447, 0.66996699669967, 0.6742718446601941, 0.675483870967742, 0.6293233082706766, 0.6573849878934624, 0.6556989247311827, 0.8574380165289256, 0.5913120567375887]
Model 2/6, Metric mean-roc_auc_score, Validation set 1: 0.678429
	best_validation_score so far: 0.680212
Fitting model 3/6
hyperparameters: {'activation': 'sigmoid', 'optimizer': 'Adam', 'momentum': 0.9, 'penalty': 0.0}


...

computed_metrics: [0.7460722702278083, 0.6113850291349171, nan, 0.6231237322515213, 0.6249243799153055, 0.6593204775022957, 0.9052631578947368, 0.6565217391304348, 0.634692962796952, 0.7124413145539905, 0.6655797101449276, 0.7307692307692308, 0.6380434782608696, 0.5840336134453782, 0.639751552795031, 0.7339357429718876, 0.6685750636132315, 0.6687992125984252, 0.6325354609929078, 0.6725601131541725, 0.6766990291262136, 0.6711827956989247, 0.6428571428571428, 0.6610169491525424, 0.6455913978494623, 0.8842975206611571, 0.6097074468085106]
Model 4/6, Metric mean-roc_auc_score, Validation set 3: 0.676911
	best_validation_score so far: 0.680212
Fitting model 5/6
hyperparameters: {'activation': 'tahn', 'optimizer': 'Adam', 'momentum': 0.9, 'penalty': 0.0}


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Only one class present in y_true. ROC AUC score is not defined in that case.
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [0.7217203456402199, 0.6203496190049306, nan, 0.6298174442190669, 0.6367211131276467, 0.6618457300275482, 0.8984962406015038, 0.6681159420289855, 0.632675930076199, 0.711267605633803, 0.6862318840579711, 0.7124260355029586, 0.6293478260869565, 0.6022408963585434, 0.65, 0.7281124497991969, 0.6825699745547074, 0.6919291338582677, 0.6281028368794326, 0.6753889674681754, 0.6847087378640777, 0.6750537634408602, 0.656390977443609, 0.6614205004035512, 0.6550537634408602, 0.8381542699724518, 0.5868794326241136]
Model 5/6, Metric mean-roc_auc_score, Validation set 4: 0.677885
	best_validation_score so far: 0.680212
Fitting model 6/6
hyperparameters: {'activation': 'tahn', 'optimizer': 'RMSprop', 'momentum': 0.9, 'penalty': 0.0}


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Only one class present in y_true. ROC AUC score is not defined in that case.
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [0.7248625294579732, 0.6131779471089198, nan, 0.6286004056795131, 0.6197822141560798, 0.6740128558310376, 0.8894736842105262, 0.6757246376811594, 0.6228148812191842, 0.7055946791862284, 0.6536231884057971, 0.7177514792899409, 0.6268115942028986, 0.5901027077497665, 0.6605590062111801, 0.7299196787148594, 0.6704834605597965, 0.6525590551181102, 0.614804964539007, 0.664073550212164, 0.6783980582524273, 0.6698924731182796, 0.6240601503759399, 0.6491121872477805, 0.6563440860215053, 0.8712121212121213, 0.5941932624113475]
Model 6/6, Metric mean-roc_auc_score, Validation set 5: 0.672229
	best_validation_score so far: 0.680212
computed_metrics: [0.917117757907183, 0.9071317586297453, 0.9628482972136223, 0.9181954348604735, 0.9337577731714539, 0.9139128589098946, 0.9666245383815567, 0.9168813166578736, 0.9176042968930869, 0.9240487651930135, 0.905302241051998, 0.9618861731325569, 0.9216900856041745, 0.9104836251963335, 0.9360144312393888, 0.9238902601473125, 0.95754554087887

In [60]:
best_dnn

MultitaskClassifier(activation_fns=None, bias_init_consts=None, dropouts=None,
                    layer_sizes=None, n_classes=2, n_features=1024, n_tasks=27,
                    weight_decay_penalty=None, weight_decay_penalty_type=None,
                    weight_init_stddevs=None)

In [61]:
best_hyperparams

('relu', 'Adam', 0.9, 0.0)

In [46]:
all_results

{"('relu', 0.9, 0.0)": 0.673746852484698,
 "('sigmoid', 0.9, 0.0)": 0.677523700567962,
 "('tahn', 0.9, 0.0)": 0.6779358768268917}

In [47]:
model=model_builder(params_dict,params_dict)

In [48]:
model.fit(train_dataset, nb_epoch=100)

154.36382479691386

In [49]:
obj=best_dnn.fit(train_dataset,**params_dict,epochs=100)

In [51]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

In [52]:
train_scores = model.evaluate(train_dataset, [metric], transformers)

computed_metrics: [0.999376477978126, 0.9997829797788218, 1.0, 0.999951157565693, 0.9999160991017668, 0.9995059613991173, 0.9998969201534545, 0.9980603916457189, 0.9999344044697566, 0.9998678200138943, 0.9997858498865583, 0.9999026407536533, 0.9998593158487091, 0.996823499074009, 0.9999209803850739, 0.9999265397273155, 0.9998704998704999, 0.9995396227942367, 0.9998608730494402, 0.9998500336910612, 0.9999192269834472, 0.9995410158848416, 0.9986063565108405, 0.9999275898194367, 0.9999651171329955, 0.9999608664175161, 0.9998507089588513]


In [53]:
valid_scores = model.evaluate(valid_dataset, [metric], transformers)

computed_metrics: [0.6904457973291438, 0.5733415508740476, nan, 0.613184584178499, 0.6409558378705384, 0.6237373737373737, 0.8082706766917294, 0.7155797101449275, 0.5750784401613627, 0.6958137715179968, 0.6271739130434784, 0.5875739644970415, 0.7090579710144927, 0.6139122315592904, 0.6487577639751553, 0.6811244979919678, 0.6596692111959288, 0.5900590551181103, 0.6015070921985816, 0.6687883074021688, 0.6182038834951455, 0.6120430107526882, 0.6736842105263159, 0.676957223567393, 0.6353225806451612, 0.7947658402203857, 0.5948581560283688]


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Only one class present in y_true. ROC AUC score is not defined in that case.
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


In [54]:
test_scores = model.evaluate(test_dataset, [metric], transformers)

computed_metrics: [0.67573385518591, 0.6939263510282161, 0.3279761904761904, 0.596551724137931, 0.7220625798212005, 0.5354109274563821, 0.6129476584022038, 0.5577673692427791, 0.6051423324150597, 0.5634050880626224, 0.6781553398058253, 0.6484444444444445, 0.5826612903225807, 0.5734295415959254, 0.6637790697674419, 0.6391397849462366, 0.765339966832504, 0.5323004201680672, 0.7463592233009708, 0.632233381157341, 0.6519742883379247, 0.611111111111111, 0.4527671755725191, 0.598995983935743, 0.6976226660877117, 0.596483942414175, 0.6470828233374133]


In [56]:
print(train_scores)
print(valid_scores)
print(test_scores)

{'mean-roc_auc_score': 0.9996075166257347}
{'mean-roc_auc_score': 0.6511487175283575}
{'mean-roc_auc_score': 0.6151409084950529}
